In [8]:
# 패키지 임포트
import requests
import pandas as pd
import numpy as np
import json
import html
import xmltodict
import re

# 지도시각화 패키지
import folium

import warnings
warnings.filterwarnings('ignore')

# 의무측정무선국에 따른 전자파 노출지수
## 맞기도하고, 아니기도 하고???

##### 지정한 위치 또는 자신의 위치 정보를 통해 해당 지역에 위치한 무선국들의 전자파 노출지수 정보를 확인
##### https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15125635

In [84]:
# 산악지역 이동통신 API
url = 'http://apis.data.go.kr/B552729/kcaApiService_emf/getEmfExposureIndex?'

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

key = ''
params = {
    'serviceKey': key,  # 인증키 / 필수값
    'address' : '전라남도 나주시 빛가람동', # 검색할 주소 또는 현재 자신의 현 위치 / 필수값
    'distance' : '1' # 입력한 주소 반경(km) 내 의무측정무선국
}

response = requests.get(url, params=params, headers=headers)
json_str = response.content.decode('utf-8')
json_object = json.loads(json_str)

In [86]:
df = pd.DataFrame(json_object['response']['body']['items']['list'])

In [88]:
# 컬럼명 변경 매핑
column_mapping = {
    'mesureRceptNo' : '측정번호', 
    'prmisnNo' : '허가번호', 
    'callNm' : '시설자', 
    'fcltyNm' : '측정기관', 
    'opertNm' : '게시물번호', 
    'mesureDe' : '측정일',
    'levelVal' : '전파측정등급', 
    'ermtwInStdrVal' : '전파기준값', 
    'mesureVal' : '전파측정값', 
    'expsrIdexVal' : '전파노출지수', 
    'ownrNm' : '시설자명',
    'gisBassAdres' : '주소', 
    'rdostDstncVal' : '무선국거리값', 
    'fqncBeginBandVal' : '주파수시작대역', 
    'fqncEndBandVal' : '주파수종료대역',
    'rdostGpsLoVal' : '경도', 
    'rdostGpsLaVal' : '위도', 
    'distance' : '무선국거리'
}

# 컬럼명 변경
df = df.rename(columns=column_mapping)

In [140]:
df['허가번호'] = df['허가번호'].str[:2] + '-' + df['허가번호'].str[2:6] + '-' + df['허가번호'].str[6:8] + '-' + df['허가번호'].str[8:]

In [118]:
# 시각화 색상설정
color = ['red', 'blue', 'green', 'purple']

In [150]:
# folium Map 생성
m = folium.Map(location=[35.0218010353365, 126.790327011452], # 나주 빛가람동 중심좌표
               zoom_start=15,
               tiles='OpenStreetMap')

for idx, val in enumerate(np.unique(df['측정기관'])):
    data = df.loc[df['측정기관'] == val, :]

    for _, row in data.iterrows():
        folium.RegularPolygonMarker(
            location=[row['위도'], row['경도']],  # 좌표
            number_of_sides=idx+3,               # 도형 꼭지점
            radius=5,                            # 지름
    
            # opacity : 투명도 0 (투명) - 1 (불투명)
            fill = True,
            fill_color= color[idx],
            fill_opacity = 1,   
                      
            # 선 색 채우기      
            color = color[idx],
            opacity = 1,
            weight = 1,
           
            # 클릭 시 팝업 문구
            tooltip = '허가번호 : ' + str(row['허가번호']) + \
            '<br>' + '측정기관 : ' + str(row['측정기관']) + \
            '<br>' + '명칭 : ' + str(row['시설자']) + \
            '<br>' + '전파측정값 : ' + str(row['전파측정값'])
      
        ).add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

#지도 띄우기
m